In [1]:
import pandas as pd
import numpy as np
#pre_processing libraries
import string
import re
from nltk.corpus import stopwords
stopwords=stopwords.words('english')
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()
import nltk
import ast
from nltk import word_tokenize, sent_tokenize, pos_tag, pos_tag_sents,RegexpParser

In [2]:
df=pd.read_csv('sample_jobs_data.csv')
df.columns

Index(['skills', 'description'], dtype='object')

In [4]:
df=df.drop(['title','jobFunction','industry','requirements'], axis = 1)

In [5]:
# take sample from data
#df=df.sample(n = 600).reset_index(drop=True)
#df=df[:500]
#save df as csv
#df.to_csv('new_sample_jobs_data.csv', index = False, header=True)
#delete nan values
df=df[df['description'] != "NaN"]
df=df.dropna()
# delete duplicated rows
df=df.drop_duplicates(subset ="description")
df.shape

(471, 2)

In [6]:
df['description']=[sent_tokenize(re.sub(r'[|]', ' ', row)) for row in df['description']]

In [7]:
# POS
tagged_texts=[]
desc_sent=[]
for row in df['description']:
    #for sent in sent_tokenize(row):
    for sent in row:
        desc_sent.append(sent)
        tagged_texts.append(pos_tag(sent.split()))

In [8]:
#chunking NPs
NPs=[]
grammar = "NP: {<NN>*|<NN>*<NNs>*|<NNs>*|<NNs>*<VBG>|<NN>*<VBG>|<VBG>|<NNP>*}"
cp  = RegexpParser(grammar)
for x in tagged_texts:
    NPs.append([' '.join([c[0] for c in chunk]) for chunk in cp.parse(x) if type(chunk) == nltk.tree.Tree]) 

In [9]:
# skills are str. ast convert string to list
df['skills']=[ast.literal_eval(y) for y in df['skills']]

#append all skills in a list
skills=[]
for x in df['skills']:
    for y in x:
        skills.append(y)
        
#unique skills      
u_skills=(np.unique(np.array(skills)))

In [10]:
def clean(row):
    r = [ " ".join(lemma.lemmatize(word.strip()) for word in s.split()
                        if word not in stopwords ) and re.sub(r'[^\w]', ' ', s.lower().strip())  for s in row ]
    
    return r

u_skills= clean(u_skills)
NPs=[clean(row) for row in NPs]
desc_sent=clean(desc_sent)

In [11]:
data = pd.DataFrame({"NPs": NPs,"description" :desc_sent})
data.head()

,NPs,description
0,"[math, create lesson, assign, homework manage...",math teachers actively instruct students crea...
1,"[accordance, principles , students , locations ]",deliver lessons in accordance with berlitz tea...
2,"[evaluation, progress]",provide evaluation for progress reports and ce...
3,"[feedback, advice , product]",provide effective feedback and advice recomme...
4,"[customer focus, delivery, duties ]",maintain an impeccable customer focus in the d...


In [12]:
#build context list with (neighbour words)n=4 for skill NPs
phrase=[]
context=[]
skill_notskill=[]
phrase_and_context=[]
for i in range(0,data.shape[0]) :
    for NP in data.NPs.iloc[i]:
        phrase.append(NP.strip())
        if NP in u_skills:
            skill_notskill.append(1)
        else:
            skill_notskill.append(0)
            
        m = re.search(r'((?:\w+\W+){,3})('+ NP.strip() +')\W+((?:\w+\W+){,3})',data.description.iloc[i]+".")
        if m:
            l = [ x.strip().split() for x in m.groups() ]
            context.append( l[0] + l[2])
            phrase_and_context.append(l[0] +l[1] +l[2])



In [13]:
data = {'phrase':phrase, 'context':context ,'phrase_and_context':phrase_and_context ,'skill_notskill':skill_notskill } 
data = pd.DataFrame(data)
data.head()

,phrase,context,phrase_and_context,skill_notskill
0,math,"[teachers, actively, instruct]","[math, teachers, actively, instruct]",1
1,create lesson,"[actively, instruct, students, plans, assign, ...","[actively, instruct, students, create, lesson,...",0
2,assign,"[create, lesson, plans, and, correct, homework]","[create, lesson, plans, assign, and, correct, ...",0
3,homework manage,"[assign, and, correct, students, in, the]","[assign, and, correct, homework, manage, stude...",0
4,classroom communicate,"[students, in, the, with, parents, and]","[students, in, the, classroom, communicate, wi...",0


In [15]:
data.to_csv('clean_data.csv', index = False, header=True)

In [16]:
data=pd.read_csv('clean_data.csv')
data.tail()

,phrase,context,phrase_and_context,skill_notskill
8384,setup,"['system', 'installation', 'and', 'and', 'cust...","['system', 'installation', 'and', 'setup', 'an...",0
8385,business,"['customize', 'them', 'for', 'needs', 'run', '...","['customize', 'them', 'for', 'business', 'need...",1
8386,system train end,"['run', 'tests', 'on', 'users', 'and', 'write']","['run', 'tests', 'on', 'system', 'train', 'end...",0
8387,instruction,"['users', 'and', 'write', 'manuals', 'as', 'ne...","['users', 'and', 'write', 'instruction', 'manu...",0
8388,necessary,"['instruction', 'manuals', 'as']","['instruction', 'manuals', 'as', 'necessary']",0


Creating words Embedding for NPs, Context and NPs + Context

In [17]:
#Create Feature Engineering with word embedding
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [18]:
import keras 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# prepare embedding matrix 
from keras.layers import Embedding
from keras.initializers import Constant

import warnings
warnings.filterwarnings('ignore')

MAX_SEQUENCE_LENGTH = 50 #max number of words in sentence
MAX_NUM_WORDS = 1000  # max number of words in vocablary
VALIDATION_SPLIT = 0.2 #ratio to split data
EMBEDDING_DIM = word_embeddings.get('a').shape[0]

Using TensorFlow backend.


In [19]:
#take list of lists text data return Embedding layer for it
def pad_sequences_data(texts):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    word_index = tokenizer.word_index
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    return word_index, data

def get_emb_layer(texts):
    word_index = pad_sequences_data(texts)[0]
    #Get Embedding vector for each word from pretraind word Embeddings(Glove)
    num_words =len(word_index) + 1
    emb_matrix = np.zeros((num_words , EMBEDDING_DIM))
    for word, i in word_index.items():
        if i > num_words:
            continue
        embedding_vector = word_embeddings.get(word) ## This references the loaded embeddings dictionary
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            emb_matrix[i] = embedding_vector
            
    # load pre-trained word embeddings into an Embedding layer
    # note that we set trainable = False so as to keep the embeddings fixed
    emb_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(emb_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
    
    return emb_layer

In [20]:
#Embedding layer for context
#convert str to list
data.context=[ast.literal_eval(x) for x in data.context]
cont_text=data.context.values.tolist()
cont_emb_layer=get_emb_layer(cont_text)


In [21]:
#Embedding layer for phrase_and_context
#convert str to list
data.phrase_and_context=[ast.literal_eval(x) for x in data.phrase_and_context]
phrase_cont_text=data.phrase_and_context.values.tolist()

emb_layer_dense=get_emb_layer(phrase_cont_text)

In [22]:
#Embedding layer for phrase
ph_text=[]
for i in range(0,data.shape[0]):
    ph_text.append(str(data.phrase.iloc[i]).strip().split())
    
phrase_emb_layer=get_emb_layer(ph_text)

In [23]:
#build model
def build_model():
    lstm_input_phrase = keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,))
    lstm_input_cont = keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,))
    dense_input = keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,))

    emb_phrase = phrase_emb_layer(lstm_input_phrase)
    lstm_emb_phrase = keras.layers.LSTM(256)(emb_phrase)
    lstm_emb_phrase = keras.layers.Dense(128, activation='relu')(lstm_emb_phrase)

    emb_cont = cont_emb_layer(lstm_input_cont)
    lstm_emb_cont = keras.layers.LSTM(256)(emb_cont)
    lstm_emb_cont = keras.layers.Dense(128, activation='relu')(lstm_emb_cont)

    dense_emb  = emb_layer_dense(dense_input)
    dense_emb = keras.layers.Dense(512, activation='relu')(dense_input)
    dense_emb = keras.layers.Dense(256, activation='relu')(dense_emb)

    keras.layers.concatenate([lstm_emb_phrase, lstm_emb_cont, dense_emb])

    x = keras.layers.concatenate([lstm_emb_phrase, lstm_emb_cont, dense_emb])
    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dense(32, activation='relu')(x)

    main_output = keras.layers.Dense(2, activation='softplus')(x)

    model = keras.models.Model(inputs=[lstm_input_phrase, lstm_input_cont, dense_input],
                                    outputs=main_output)

    optimizer = keras.optimizers.Adam(lr=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy')

    return model
    
'''onehot_transformfunction 
   converts the target values 0 and 1 to one-hot vectors [1,0] and [0,1]'''

def onehot_transform(y):

    onehot_y = []

    for numb in y:
        onehot_arr = np.zeros(2)
        onehot_arr[numb] = 1
        onehot_y.append(np.array(onehot_arr))

    return np.array(onehot_y)

In [24]:

#fit the model
def fit( x_lstm_phrase, x_lstm_context, x_dense, y,
            val_split=0.25, patience=5, max_epochs=1000, batch_size=32):
    
        x_lstm_phrase_seq = pad_sequences_data(x_lstm_phrase)[1]
        x_lstm_context_seq = pad_sequences_data(x_lstm_context)[1]
        x_dense_seq=pad_sequences_data(x_dense)[1]
        y_onehot=onehot_transform(y)
        model.fit([x_lstm_phrase_seq, x_lstm_context_seq, x_dense_seq] ,y_onehot,
                   batch_size=batch_size,
                   epochs = max_epochs ,
                   validation_split=val_split,
                   callbacks=[keras.callbacks.EarlyStopping
                              (monitor='val_loss', patience=patience)])



In [25]:
model= build_model()
fit(ph_text,cont_text,phrase_cont_text,data['skill_notskill'].values.tolist())





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 6291 samples, validate on 2098 samples
Epoch 1/1000





6291/6291 [==============================] - 60s 9ms/step - loss: 2.1875 - val_loss: 2.1636
Epoch 2/1000
6291/6291 [==============================] - 55s 9ms/step - loss: 1.7614 - val_loss: 1.8722
Epoch 3/1000
6291/6291 [==============================] - 59s 9ms/step - loss: 1.8003 - val_loss: 2.3835
Epoch 4/1000
6291/6291 [==============================] - 60s 9ms/step - loss: 1.7219 - val_loss: 1.5090
Epoch 5/1000
6291/6291 [==============================] - 61s 10ms/step - loss: 1.4522 - val_loss: 1.4214
Epoch 6/1000
6291/6291 [==============================] - 65s 10ms/step - loss: 1.5308 - val_loss: 1.5560
Epoch 7/1000
6291/6291 [==============================] - 67s 11ms/step - loss: 1.6163 - val_loss: 2.0305
Epoch 8/1000
6291/6291 [==============================] - 69s 11ms/step - loss: 1.7029 - val_loss: 1.5452
Ep